# ECE analysis

In [60]:
import utils.utils as utils

from utils.constants import CIFAR10_DICT_MODELS_TO_ID, IMAGENET_DICT_MODELS_TO_ID

import plot_results as ps

In [61]:
robustness_level = "robust"
ds_name = "cifar10"

ece_save_path = "figures/calibration_hist/"
ece_dict_name = f"calibration_curve_{robustness_level}_{ds_name}"
ece_dict = utils.my_load(f"{ece_save_path}{ece_dict_name}.pkl")

In [62]:
entropy_save_path = "figures/violin_plots/"
entropy_dict_name = f"violinplots_{robustness_level}_{ds_name}"
entropy_dict = utils.my_load(f"{entropy_save_path}{entropy_dict_name}.pkl")

In [63]:
dict_to_model = CIFAR10_DICT_MODELS_TO_ID if ds_name == "cifar10" else IMAGENET_DICT_MODELS_TO_ID

In [64]:
# print("ECE")
# for idx, val in ece_dict.items():
#     print(CIFAR10_DICT_MODELS_TO_ID[idx], end=" ")
#     for idx, val in val.items():
#         print(f"& {val:.3f}", end=" ")
#         
#     print("\\\\")

In [65]:
atk_type = "under_ece"
attack_ece = {k: v[atk_type] for k, v in ece_dict.items()}
sort_attack = dict(sorted(attack_ece.items(), key=lambda item: item[1]))

# print(atk_type)
# counter = 0
# for idx, item in sort_attack.items():
#     print(f"{idx} --> {item}")
#     counter += item
#     
# print(counter/len(sort_attack))

In [66]:
atk_type = "over_ece"
attack_ece = {k: v[atk_type] for k, v in ece_dict.items()}
sort_attack = dict(sorted(attack_ece.items(), key=lambda item: item[1]))

# print(atk_type)
# counter = 0
# for idx, item in sort_attack.items():
#     print(f"{idx} --> {item}")
#     counter += item
#     
# print(counter/len(sort_attack))

In [67]:
# ECE MAX DIFFERENCE Uatk and Oatk
difference_ece_dict = {k: abs(v["under_ece"]-v["clean_ece"]) + abs(v["clean_ece"]-v["over_ece"]) for k, v in ece_dict.items()}
sort_difference = dict(sorted(difference_ece_dict.items(), key=lambda item: item[1]))

# print("ECE delta")
# 
# for idx, item in sort_difference.items():
#     print(f"{idx} --> {item}")

### ECE table all metrics (Clean - Over - Umder)

Per formattare bene i numeri
$\displaystyle 0.222\genfrac{}{}{-.4pt}{}{+0.052}{-0.095}$

In [68]:
clean_ece = {k: v["clean_ece"] for k, v in ece_dict.items()}
under_ece = {k: v["under_ece"] for k, v in ece_dict.items()}
over_ece = {k: v["over_ece"] for k, v in ece_dict.items()}

In [69]:
clean_entropy = {k: v["clean"] for k, v in entropy_dict.items()}
under_entropy = {k: v["uatk"] for k, v in entropy_dict.items()}
over_entropy = {k: v["oatk"] for k, v in entropy_dict.items()}

In [70]:
# Displacement
# for idx, item in clean_ece.items():
#     print(f"{CIFAR10_DICT_MODELS_TO_ID[idx]} --> clean: {item:.3f} under: +{(under_ece[idx]-item):.3f} over: -{(item - over_ece[idx]):.3f}")

In [71]:
# Upper - lower - Delta
# for idx, item in clean_ece.items():
#     print(f"{CIFAR10_DICT_MODELS_TO_ID[idx]} --> clean: {item:.3f} under: {(under_ece[idx]):.3f} over: {(over_ece[idx]):.3f} diff: {(under_ece[idx] - over_ece[idx]):.3f}")

In [72]:
# # Print table single column
# print("""
# \\bgroup
# \def\\arraystretch{1.7}
# \\begin{table}[]
# \\resizebox{\\textwidth}{!}{
# \\begin{tabular}{lllll}
# & \multicolumn{1}{c}{ECE} & \multicolumn{1}{c}{ECE Delta} & \multicolumn{1}{c}{Entropy} & \multicolumn{1}{c}{Entropy Delta} \\\\""")
# 
# for idx, item in clean_ece.items():
#     clean_ece_value = item.round(3)
#     under_ece_value = f"+{(under_ece[idx] - item):.3f}" # under_ece_value = f"+{under_ece[idx]}"
#     over_ece_value = f"-{(item - over_ece[idx]):.3f}"   # over_ece_value = f"-{over_ece[idx]}"
#     
#     ece_equation = f"$\displaystyle {clean_ece_value}\genfrac{{}}{{}}{{-.4pt}}{{}}{{{under_ece_value}}}{{{over_ece_value}}}$"
#     ece_diff = f"{(under_ece[idx] - over_ece[idx]):.3f}"
#     entropy_equation = ""
#     entropy_diff = ""
#     print(f"{dict_to_model[idx]} & {ece_equation} & ${ece_diff}$ & entropy & diff \\\\")
#     
# print("""\end{tabular}}
# \caption{}
# \label{}
# \end{table}
# \egroup""")

In [73]:
# Print table two columns
print("""
\\bgroup
\def\\arraystretch{1.7}
\\begin{table}[]
\\resizebox{\\textwidth}{!}{%
\\begin{tabular}{llllllllll}
   & \multicolumn{1}{c}{ECE} & \multicolumn{1}{c}{Delta} & \multicolumn{1}{c}{Entropy} & \multicolumn{1}{c}{Delta} &    & \multicolumn{1}{c}{ECE} & \multicolumn{1}{c}{Delta} & \multicolumn{1}{c}{Entropy} & \multicolumn{1}{c}{Delta} \\\\""")

for n, (idx, item) in enumerate(clean_ece.items()):
    # ECE
    clean_ece_value = item.round(3)
    under_ece_value = f"+{(under_ece[idx] - item):.3f}" # under_ece_value = f"+{under_ece[idx]}"
    over_ece_value = f"-{(item - over_ece[idx]):.3f}"   # over_ece_value = f"-{over_ece[idx]}"
    ece_equation = f"$\displaystyle {clean_ece_value}\genfrac{{}}{{}}{{-.4pt}}{{}}{{{under_ece_value}}}{{{over_ece_value}}}$"
    ece_diff = f"{(under_ece[idx] - over_ece[idx]):.3f}"
    
    # ENTROPY
    clean_entropy_value = round(clean_entropy[idx],3)
    under_entropy_value = f"+{(under_entropy[idx] - clean_entropy_value):.3f}" # under_entropy_value = f"+{under_entropy[idx]}"
    over_entropy_value = f"-{(clean_entropy_value - over_entropy[idx]):.3f}"   # over_entropy_value = f"-{over_entropy[idx]}"
    entropy_equation = f"$\displaystyle {clean_entropy_value}\genfrac{{}}{{}}{{-.4pt}}{{}}{{{under_entropy_value}}}{{{over_entropy_value}}}$"
    entropy_diff = f"{(under_entropy[idx] - over_entropy[idx]):.3f}"
    
    print(f"{dict_to_model[idx]} & {ece_equation} & ${ece_diff}$ & {entropy_equation} & {entropy_diff}", end=" & " if (n+1)%2 != 0 else " \\\\ \n")
    
print("""
\end{tabular}}
\caption{}
\label{}
\end{table}
\egroup
""")


\bgroup
\def\arraystretch{1.7}
\begin{table}[]
\resizebox{\textwidth}{!}{%
\begin{tabular}{llllllllll}
   & \multicolumn{1}{c}{ECE} & \multicolumn{1}{c}{Delta} & \multicolumn{1}{c}{Entropy} & \multicolumn{1}{c}{Delta} &    & \multicolumn{1}{c}{ECE} & \multicolumn{1}{c}{Delta} & \multicolumn{1}{c}{Entropy} & \multicolumn{1}{c}{Delta} \\
\addepallieffID & $\displaystyle 0.387\genfrac{}{}{-.4pt}{}{+0.052}{-0.095}$ & $0.146$ & $\displaystyle 1.625\genfrac{}{}{-.4pt}{}{+0.180}{-0.155}$ & 0.335 & \addepallitowID & $\displaystyle 0.259\genfrac{}{}{-.4pt}{}{+0.070}{-0.134}$ & $0.203$ & $\displaystyle 1.269\genfrac{}{}{-.4pt}{}{+0.270}{-0.268}$ & 0.539 \\ 
\sehwagrobustID & $\displaystyle 0.141\genfrac{}{}{-.4pt}{}{+0.105}{-0.119}$ & $0.224$ & $\displaystyle 0.877\genfrac{}{}{-.4pt}{}{+0.349}{-0.300}$ & 0.649 & \engstromID & $\displaystyle 0.094\genfrac{}{}{-.4pt}{}{+0.125}{-0.039}$ & $0.164$ & $\displaystyle 0.618\genfrac{}{}{-.4pt}{}{+0.460}{-0.361}$ & 0.821 \\ 
\sehwagrobustIDdue & $\displa

## Latex table type 2


In [85]:
model_x_row = 8
total_model = len(clean_ece.items())

print("""\\begin{table}[]
\\resizebox{\\textwidth}{!}{%
\\begin{tabular}{lllllllllll}""")

tot_rows = total_model//model_x_row + 1 if total_model%model_x_row != 0 else 0


for row in range(tot_rows):    
    # intestazione
    model_names = list(dict_to_model.values())
    print("&  ", end=" ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& \multicolumn{{1}}{{c}}{{{model_names[idx]}}}", end=" ")
    print(f"\\\\ \cdashline{{3-{model_x_row+2}}}")
    
    # UNDER ATK
    under_ece_values = list(under_ece.values())
    print("\multicolumn{1}{c}{\multirow{3}{*}{ECE}}  & Otk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {under_ece_values[idx]:.3f}", end=" ")
    print("\\\\")
    
    # CLEAN
    clean_ece_values = list(clean_ece.values())
    print("\multicolumn{1}{c}{} & Clean  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {clean_ece_values[idx]:.3f}", end=" ")
    print("\\\\")
    
    
    # OVER
    over_ece_values = list(over_ece.values())
    print("\multicolumn{1}{c}{} & Oatk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {over_ece_values[idx]:.3f}", end=" ")
    
    print(f"\\\\ \cline{{1-{model_x_row+2}}}")
    
    
    # ENTROPYYYYYYYYYYYYYYYYYYYYYYY
    # UNDER ATK
    under_entropy_values = list(under_entropy.values())
    print("\multicolumn{1}{c}{\multirow{3}{*}{H(x)}}  & Otk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {under_entropy_values[idx]:.3f}", end=" ")
    print("\\\\")
    
    # CLEAN
    clean_entropy_values = list(clean_entropy.values())
    print("\multicolumn{1}{c}{} & Clean  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {clean_entropy_values[idx]:.3f}", end=" ")
    print("\\\\")
    
    
    # OVER
    over_entropy_values = list(over_entropy.values())
    print("\multicolumn{1}{c}{} & Oatk  ", end= " ")
    for i in range(model_x_row):
        idx = i+(row*model_x_row)
        if idx >= total_model:
            break
        print(f"& {over_entropy_values[idx]:.3f}", end=" ")
    
    print(f"\\\\ \cline{{1-{model_x_row+2}}}")
    
    

print("\end{tabular}}\n\end{table}")

\begin{table}[]
\resizebox{\textwidth}{!}{%
\begin{tabular}{lllllllllll}
&   & \multicolumn{1}{c}{\addepallieffID} & \multicolumn{1}{c}{\addepallitowID} & \multicolumn{1}{c}{\sehwagrobustID} & \multicolumn{1}{c}{\engstromID} & \multicolumn{1}{c}{\sehwagrobustIDdue} & \multicolumn{1}{c}{\cuidecoupledID} & \multicolumn{1}{c}{\gowalimprovingID} & \multicolumn{1}{c}{\wangbetterID} \\ \cdashline{3-10}
\multicolumn{1}{c}{\multirow{3}{*}{ECE}}  & Otk   & 0.439 & 0.328 & 0.246 & 0.219 & 0.193 & 0.392 & 0.186 & 0.366 \\
\multicolumn{1}{c}{} & Clean   & 0.387 & 0.259 & 0.141 & 0.094 & 0.108 & 0.321 & 0.096 & 0.294 \\
\multicolumn{1}{c}{} & Oatk   & 0.293 & 0.125 & 0.022 & 0.055 & 0.012 & 0.244 & 0.020 & 0.213 \\ \cline{1-10}
\multicolumn{1}{c}{\multirow{3}{*}{H(x)}}  & Otk   & 1.805 & 1.539 & 1.226 & 1.078 & 0.966 & 1.550 & 0.961 & 1.480 \\
\multicolumn{1}{c}{} & Clean   & 1.625 & 1.269 & 0.877 & 0.618 & 0.645 & 1.355 & 0.621 & 1.262 \\
\multicolumn{1}{c}{} & Oatk   & 1.470 & 1.001 & 0.577 & 0.2

# Entropy

In [76]:
print(entropy_dict)

{'addepalli2022': {'clean': 1.6248085498809814, 'oatk': 1.4698271751403809, 'uatk': 1.805284857749939}, 'addepalli2022_towards': {'clean': 1.2685953378677368, 'oatk': 1.0006461143493652, 'uatk': 1.539366364479065}, 'sehwag2021': {'clean': 0.8774853944778442, 'oatk': 0.5769382119178772, 'uatk': 1.2257531881332397}, 'engstrom2019': {'clean': 0.6181110739707947, 'oatk': 0.25676676630973816, 'uatk': 1.0777428150177002}, 'sehwag2021Proxy_ResNest152': {'clean': 0.6446627974510193, 'oatk': 0.37193426489830017, 'uatk': 0.9663532972335815}, 'Cui2023Decoupled_WRN_28_10': {'clean': 1.3550007343292236, 'oatk': 1.203749656677246, 'uatk': 1.5503093004226685}, 'gowal2021Improving_28_10': {'clean': 0.6213864088058472, 'oatk': 0.34705400466918945, 'uatk': 0.9607109427452087}, 'Wang2023Better_WRN_28_10': {'clean': 1.261987328529358, 'oatk': 1.0996037721633911, 'uatk': 1.4796520471572876}, 'Xu2023Exploring_WRN_28_10': {'clean': 0.8879414200782776, 'oatk': 0.5057478547096252, 'uatk': 1.4252583980560303}, 

In [77]:
for idx, val in entropy_dict.items():
    print(dict_to_model[idx], end=" ")
    for idx, val in val.items():
        print(f"& {val:.3f}", end=" ")
        
    print("\\\\")

\addepallieffID & 1.625 & 1.470 & 1.805 \\
\addepallitowID & 1.269 & 1.001 & 1.539 \\
\sehwagrobustID & 0.877 & 0.577 & 1.226 \\
\engstromID & 0.618 & 0.257 & 1.078 \\
\sehwagrobustIDdue & 0.645 & 0.372 & 0.966 \\
\cuidecoupledID & 1.355 & 1.204 & 1.550 \\
\gowalimprovingID & 0.621 & 0.347 & 0.961 \\
\wangbetterID & 1.262 & 1.100 & 1.480 \\
\xuexploringID & 0.888 & 0.506 & 1.425 \\
\gowalimprovingIDdue & 0.562 & 0.317 & 0.872 \\
\kangstableID & 0.179 & 0.082 & 0.231 \\
\pangrobustnessID & 0.957 & 0.751 & 1.221 \\
\rebuffifixingID & 0.968 & 0.731 & 1.291 \\
\wangbetterIDdue & 1.192 & 1.036 & 1.410 \\
\pengrobustID & 1.181 & 1.029 & 1.396 \\


# Latex graph grid

In [78]:
import glob

plots_path = glob.glob("./figures/calibration_hist/cifar10/*")

cols = 3


print("\\begin{tabular}{"+"l"*cols+"}")
for id, path in enumerate(plots_path):

    print(f"\includegraphics[width={(1/cols - 0.01):.2f}\\linewidth]"+"{"+path[2:]+"}", end = " ")
    
    if id != 0 and (id+1) % cols == 0:
        print("\\\\")
    else:
        print(" &")

print("\end{tabular}")

\begin{tabular}{lll}
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_addepalli2022_towards.pdf}  &
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_sehwag2021Proxy_ResNest152.pdf}  &
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_sehwag2021.pdf} \\
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_Gowal2021Improving_70_16_ddpm_100m.pdf}  &
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_Wang2023Better_WRN_70_16.pdf}  &
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_Cui2023Decoupled_WRN_28_10.pdf} \\
\includegraphics[width=0.32\linewidth]{figures/calibration_hist/cifar10/calibration_curve_robust_cifar10_addepalli2022.pdf}  &
\includegraphics[width=0.32